In [2]:
import pandas as pd
import numpy as np
import os
import json

In [57]:
meta = pd.read_parquet('datasets\\Google Maps\\metadata-sitios\\all_in_one.parquet')

In [ ]:
# En este bloque, realize la apertura de todos los .json de meta datos de google, para luego concatenarlos y formar un solo dataframe con toda la informacion

'''
dicc = {}

for x in range(1,12):
    dicc[x] = pd.read_json(f'datasets\\Google Maps\\metadata-sitios\\{x}.json', lines=True)
    
    
# Lista para almacenar los DataFrames
lista_dataframes = []

# Iterar sobre los valores del diccionario (los DataFrames)
for df in dicc.values():
    lista_dataframes.append(df)

# Concatenar los DataFrames en uno solo
resultado_concatenado = pd.concat(lista_dataframes, ignore_index=True)


resultado_concatenado.to_parquet(r'datasets\Google Maps\metadata-sitios\all_in_one.parquet',index=False)

'''

In [99]:
# Desanido todas categorias por cada establecimiento
expanded_categories = meta['category'].explode().dropna()

# Contar la cantidad de cada categoría
cantidad_por_categoria = expanded_categories.value_counts()

In [101]:
# Creo un filtro de la cantidad de locales que corresponden a cada categoria, el fin de esta accion es visualizar el total de cada categoria para seleccionar el tipo de comercio que vamos a utilizar en el proyecto.
categorias_mas_de_30 = cantidad_por_categoria[cantidad_por_categoria > 30000]

print(categorias_mas_de_30)

category
Restaurant               97257
Auto repair shop         77854
Gas station              70143
Service establishment    66476
Beauty salon             65717
Convenience store        62165
Church                   53069
Hair salon               51765
Nail salon               44597
ATM                      42452
Park                     42414
Doctor                   36669
Clothing store           36470
Dentist                  36138
Insurance agency         35677
Barber shop              35325
Grocery store            31699
Bar                      30931
Cell phone store         30584
Name: count, dtype: int64


## Columnas a eliminar

-state (por que no es relevante, dice si esta habierto el local o no en base al dia y )

-url (se puede ingresar utilizando la siguiente url mas el gmap-id) [https://www.google.com/maps/place//data=!4m2!3m1!1s {aca va el gmap_id}]

-description (esta informacion podemos obtenerla en caso de ser necesario usando la api, solo tiene 250.000 datos lo que vuelve a la columna irrelevante en comparacion con la cantidad total, por eso mismo se recomienda realizar la extraccion de dicha columna de ser necesario desde la api)

-MISC (esta columna tenia una descripcion de como estan las condiciones del local)

-price (es irrelevante el precio)

-num_of_reviews (esta columna tendremos que volver a crearla luego de realizar el etl de los estados)

In [85]:
meta.drop(columns=['state','url','description','MISC','price','num_of_reviews'],inplace=True)

In [94]:
meta

,name,address,gmap_id,latitude,longitude,category,avg_rating,hours,relative_results
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,[Pharmacy],4.9,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1..."
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.215290,[Textile exporter],4.5,None,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2..."
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,[Korean restaurant],4.4,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2..."
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,34.023669,-118.232930,[Fabric store],3.3,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2..."
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,34.036694,-118.249421,[Fabric store],4.3,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2..."
...,...,...,...,...,...,...,...,...,...
3025006,Steak 48,"Steak 48, 260 S Broad St, Philadelphia, PA 19102",0x89c6c74f43a49b55:0x6be6995921c58b12,39.947254,-75.164953,"[Steak house, Fine dining restaurant, Seafood ...",4.6,"[[Monday, 4–10PM], [Tuesday, 4–10PM], [Wednesd...","[0x89c6c7af1a55f4f9:0x589bb93addf7fd95, 0x89c6..."
3025007,"Jack Mcnerney Chevrolet, Inc.","Jack Mcnerney Chevrolet, Inc., 363 NY-281, Tul...",0x89da1787d3fdc4a5:0xce92ea6e3cd4d1cc,42.788636,-76.122120,[Chevrolet dealer],4.4,"[[Monday, 8:30AM–7PM], [Tuesday, 8:30AM–7PM], ...","[0x89d98d228469e435:0x3fdc4fa72594151, 0x89d9e..."
3025008,Central Ny Spay Neuter Assista,"Central Ny Spay Neuter Assista, 17 Salisbury S...",0x89da6b9ce832726f:0xbd0d3630f1e3b42e,42.603831,-76.165286,[Veterinarian],4.3,None,"[0x89da6c67512700cb:0x69892b3a9a648efe, 0x89d9..."
3025009,Ok Feed Store,"Ok Feed Store, 16300 SW 296th St, Homestead, F...",0x88d9e872a9e39f3d:0x5de29f50507e5d73,25.491641,-80.456076,[Animal feed store],4.9,"[[Monday, 9AM–6PM], [Tuesday, 9AM–6PM], [Wedne...","[0x88d9e86cb955da7b:0xeda838b28926c524, 0x88d9..."


In [93]:
# En este bloque exporto nuevamente el parquet con las columnas eliminadas

# meta.to_parquet(r'datasets\Google Maps\metadata-sitios\all_in_one_limpio.parquet',index=False)